In [1]:
from __future__ import division, print_function, absolute_import
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
import numpy
import PIL
from PIL import Image
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
from keras.optimizers import RMSprop
from keras import backend as K
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

/home/aniruddha/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [42]:
learning_rate = 0.01
num_steps = 20
batch_size = 20
total_numbers = 291
display_step = 1000
examples_to_show = 10

# Network Parameters
num_hidden_1 = 128 # 1st layer num features
num_hidden_2 = 32 # 2nd layer num features (the latent dim)
num_input = 42 
num_classes = 5

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])


weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
    'classifier_h': tf.Variable(tf.random_normal([num_hidden_2, num_classes])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([num_input])),
    'classifier_b': tf.Variable(tf.random_normal([num_classes])),
}

In [43]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op

label_pred = tf.nn.softmax(tf.add(tf.matmul(encoder_op, weights['classifier_h']), biases['classifier_b']))
y_clipped = tf.clip_by_value(label_pred, 1e-10, 0.9999999)


# Targets (Labels) are the input data.
y_true = X
label_true = Y

# Define loss and optimizer, minimize the squared error
loss_autoencoder = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
cross_entropy_loss = -tf.reduce_mean(tf.reduce_sum(label_true * tf.log(y_clipped)
                         + (1 - label_true) * tf.log(1 - y_clipped), axis=1))
loss_total = loss_autoencoder+cross_entropy_loss

optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss_total)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [4]:
from numpy import genfromtxt
my_data = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VHc.csv', delimiter=',')
my_data1 = genfromtxt('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/Cgamma_VVc.csv', delimiter=',')

In [15]:
data = my_data[1:,24:]
data1 = my_data1[1:,24:]
print(data.shape)
print(data.shape)

(291, 21)
(291, 21)


In [16]:
print(data.max())

0.142512947320938


In [17]:
data = np.array(data)
data1 = np.array(data1)
print(data.shape)
print(data1.shape)

(291, 21)
(291, 21)


In [9]:
import pandas
colnames = ['Latitude', 'Longitude', 'CType', 'Remarks', 'Index']
data1 = pandas.read_csv('/home/aniruddha/Documents/deep_learning_papers/DataBiplab/CropClean.csv', names=colnames)

In [10]:
ctype = data1.CType.tolist()
print(ctype)

['CType', 'PD', 'PD', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'SC', 'SC', 'SC', 'JW', 'PD', 'SC', 'BA', 'SC', 'BA', 'PD', 'PD', 'SC', 'PP', 'PD', 'PD', 'PD', 'PD', 'PD', 'BA', 'SC', 'SC', 'BA', 'BA', 'JW', 'JW', 'SC', 'PD', 'BA', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'BA', 'SC', 'PD', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'PD', 'PD', 'SC', 'JW', 'PD', 'PD', 'BA', 'BA', 'SC', 'SC', 'SC', 'BA', 'PP', 'PP', 'BA', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PP', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'JW', 'SC', 'PD', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'BA', 'BA', 'PD', 'PD', 'SC', 'PD', 'SC', 'PD', 'BA', 'JW', 'PD', 'SC', 'JW', 'SC', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'PD', 'SC', 'SC', 'PD', 'SC', 'JW', 'SC', 'SC', 'SC', 'SC', 'SC', 'SC', 'PD', 'JW', 'PD', 'SC', 'SC', 'JW', 

In [11]:
p = np.array(ctype)
print(p.shape)

(292,)


In [12]:
lbl1 = ctype
label = []
for i in range(1,292):
    if(lbl1[i] == "PD"):
        label += [0]
    if(lbl1[i] == "JW"):
        label += [1]
    if(lbl1[i] == "PP"):
        label += [2]
    if(lbl1[i] == "SC"):
        label += [3]
    if(lbl1[i] == "BA"):
        label += [4]
print(label)

[0, 0, 0, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 1, 0, 3, 4, 3, 4, 0, 0, 3, 2, 0, 0, 0, 0, 0, 4, 3, 3, 4, 4, 1, 1, 3, 0, 4, 1, 3, 3, 3, 3, 3, 0, 0, 4, 3, 0, 0, 3, 3, 3, 3, 3, 1, 0, 0, 3, 1, 0, 0, 4, 4, 3, 3, 3, 4, 2, 2, 4, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 0, 0, 3, 0, 3, 0, 4, 1, 0, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 1, 3, 3, 3, 3, 3, 3, 0, 1, 0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 0, 4, 4, 3, 0, 4, 0, 3, 3, 3, 3, 0, 3, 0, 3, 2, 3, 0, 3, 1, 4, 3, 0, 0, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 4, 1, 0, 0, 2, 3, 3, 3, 0, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 0, 4, 4, 4, 0, 0, 0, 0, 3, 3, 3, 3, 4, 3, 3, 4, 4, 3, 3, 1, 3, 4, 1, 3, 3, 3, 3, 4, 0, 0]


In [13]:
label = np.array(label)
print(label.shape)

(291,)


In [18]:
data2 = np.concatenate((data,data1), axis=1)
print(data2.shape)

(291, 42)


In [44]:
# 80% train 20% test (NDVIclean)
X_train = data2[0:232,:]
X_test = data2[232:,:]
y_train = label[0:232]
y_test = label[232:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(232, 42)
(59, 42)
(232,)
(59,)


In [45]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(232, 5)
(59, 5)


In [46]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 13

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 0.78705116
Epoch: 2 cost = 0.61961025
Epoch: 3 cost = 0.50568744
Epoch: 4 cost = 0.45082413
Epoch: 5 cost = 0.43016690
Epoch: 6 cost = 0.42252247
Epoch: 7 cost = 0.41902740
Epoch: 8 cost = 0.41682220
Epoch: 9 cost = 0.41504975
Epoch: 10 cost = 0.41344120
Epoch: 11 cost = 0.41189704
Epoch: 12 cost = 0.41037341
Epoch: 13 cost = 0.40884999
Epoch: 14 cost = 0.40731718
Epoch: 15 cost = 0.40576850
Epoch: 16 cost = 0.40419531
Epoch: 17 cost = 0.40258318
Epoch: 18 cost = 0.40090950
Epoch: 19 cost = 0.39914155
Epoch: 20 cost = 0.39723651
Epoch: 21 cost = 0.39514336
Epoch: 22 cost = 0.39280972
Epoch: 23 cost = 0.39019249
Epoch: 24 cost = 0.38727076
Epoch: 25 cost = 0.38405633
Epoch: 26 cost = 0.38060021
Epoch: 27 cost = 0.37699322
Epoch: 28 cost = 0.37336071
Epoch: 29 cost = 0.36983384
Epoch: 30 cost = 0.36648807
Epoch: 31 cost = 0.36328398
Epoch: 32 cost = 0.36008925
Epoch: 33 cost = 0.35677791
Epoch: 34 cost = 0.35329833
Epoch: 35 cost = 0.34966089
Epoch: 36 cost = 0.34589793
E

Epoch: 413 cost = 0.00413935
Epoch: 414 cost = 0.00411388
Epoch: 415 cost = 0.00408937
Epoch: 416 cost = 0.00408818
Epoch: 417 cost = 0.00413326
Epoch: 418 cost = 0.00416405
Epoch: 419 cost = 0.00414446
Epoch: 420 cost = 0.00416011
Epoch: 421 cost = 0.00406121
Epoch: 422 cost = 0.00400870
Epoch: 423 cost = 0.00399576
Epoch: 424 cost = 0.00399844
Epoch: 425 cost = 0.00399037
Epoch: 426 cost = 0.00407943
Epoch: 427 cost = 0.00408812
Epoch: 428 cost = 0.00416899
Epoch: 429 cost = 0.00406067
Epoch: 430 cost = 0.00402351
Epoch: 431 cost = 0.00397960
Epoch: 432 cost = 0.00408889
Epoch: 433 cost = 0.00406784
Epoch: 434 cost = 0.00392364
Epoch: 435 cost = 0.00392778
Epoch: 436 cost = 0.00406195
Epoch: 437 cost = 0.00405853
Epoch: 438 cost = 0.00412140
Epoch: 439 cost = 0.00592378
Epoch: 440 cost = 0.26609125
Epoch: 441 cost = 0.02649489
Epoch: 442 cost = 0.00485670
Epoch: 443 cost = 0.00439786
Epoch: 444 cost = 0.00424839
Epoch: 445 cost = 0.00410535
Epoch: 446 cost = 0.00406402
Epoch: 447 cos

Epoch: 791 cost = 0.00366272
Epoch: 792 cost = 0.00367390
Epoch: 793 cost = 0.00364631
Epoch: 794 cost = 0.00364501
Epoch: 795 cost = 0.00360187
Epoch: 796 cost = 0.00357137
Epoch: 797 cost = 0.00365873
Epoch: 798 cost = 0.00364138
Epoch: 799 cost = 0.00369931
Epoch: 800 cost = 0.00367609
Epoch: 801 cost = 0.00362599
Epoch: 802 cost = 0.00368629
Epoch: 803 cost = 0.00367399
Epoch: 804 cost = 0.00364133
Epoch: 805 cost = 0.00362115
Epoch: 806 cost = 0.00361325
Epoch: 807 cost = 0.00362104
Epoch: 808 cost = 0.00358664
Epoch: 809 cost = 0.00354824
Epoch: 810 cost = 0.00365956
Epoch: 811 cost = 0.00362900
Epoch: 812 cost = 0.00367334
Epoch: 813 cost = 0.00367807
Epoch: 814 cost = 0.00360713
Epoch: 815 cost = 0.00368051
Epoch: 816 cost = 0.00357808
Epoch: 817 cost = 0.00359430
Epoch: 818 cost = 0.00360407
Epoch: 819 cost = 0.00371083
Epoch: 820 cost = 0.00372614
Epoch: 821 cost = 0.00364071
Epoch: 822 cost = 0.00357241
Epoch: 823 cost = 0.00359143
Epoch: 824 cost = 0.00365922
Epoch: 825 cos

In [47]:
# define an accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(label_pred, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [48]:
print(y_test.shape)

(59, 5)


In [49]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.7288136


In [27]:
# 70% train 30% test (NDVIclean)
# Train - 204
# Test - 87
X_train = data2[0:204,:]
X_test = data2[204:,:]
y_train = label[0:204]
y_test = label[204:]

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(204, 42)
(87, 42)
(204,)
(87,)


In [28]:
from keras.utils import np_utils
y_test = np_utils.to_categorical(y_test)
y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
print(y_test.shape)

(204, 5)
(87, 5)


In [31]:
# Start Training
# Start a new TF session
sess = tf.Session()

# Run the initializer
sess.run(init)
batch_size = 17
num_batch = 12

# Training
for i in range(0,1000):
    k = 0 
    # Prepare Data
    # Get the next batch of MNIST data (only images are needed, not labels)
    avg_cost = 0
    for j in (0,num_batch):
        batch_x = X_train[k:k+batch_size,:]
        batch_y = y_train[k:k+batch_size,:]
        k += 29

    # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss_total], feed_dict={X: batch_x, Y: batch_y})
        avg_cost += l / num_batch
    print("Epoch:", (i + 1), "cost =", "{:.8f}".format(avg_cost))

Epoch: 1 cost = 1.11544287
Epoch: 2 cost = 0.95239111
Epoch: 3 cost = 0.79871809
Epoch: 4 cost = 0.67495094
Epoch: 5 cost = 0.60351582
Epoch: 6 cost = 0.57520086
Epoch: 7 cost = 0.56272473
Epoch: 8 cost = 0.55302219
Epoch: 9 cost = 0.54372223
Epoch: 10 cost = 0.53474150
Epoch: 11 cost = 0.52602983
Epoch: 12 cost = 0.51743708
Epoch: 13 cost = 0.50884996
Epoch: 14 cost = 0.50023303
Epoch: 15 cost = 0.49158333
Epoch: 16 cost = 0.48287328
Epoch: 17 cost = 0.47403445
Epoch: 18 cost = 0.46498054
Epoch: 19 cost = 0.45565126
Epoch: 20 cost = 0.44606394
Epoch: 21 cost = 0.43637987
Epoch: 22 cost = 0.42699061
Epoch: 23 cost = 0.41853680
Epoch: 24 cost = 0.41165799
Epoch: 25 cost = 0.40654816
Epoch: 26 cost = 0.40284344
Epoch: 27 cost = 0.39994140
Epoch: 28 cost = 0.39729949
Epoch: 29 cost = 0.39453640
Epoch: 30 cost = 0.39143012
Epoch: 31 cost = 0.38788768
Epoch: 32 cost = 0.38391197
Epoch: 33 cost = 0.37957269
Epoch: 34 cost = 0.37498039
Epoch: 35 cost = 0.37025567
Epoch: 36 cost = 0.36549038
E

Epoch: 294 cost = 0.00553104
Epoch: 295 cost = 0.00559265
Epoch: 296 cost = 0.00554167
Epoch: 297 cost = 0.00559942
Epoch: 298 cost = 0.00568552
Epoch: 299 cost = 0.00588095
Epoch: 300 cost = 0.00611536
Epoch: 301 cost = 0.00641214
Epoch: 302 cost = 0.00670746
Epoch: 303 cost = 0.00730763
Epoch: 304 cost = 0.00825148
Epoch: 305 cost = 0.00951568
Epoch: 306 cost = 0.04281384
Epoch: 307 cost = 0.12980730
Epoch: 308 cost = 0.01282324
Epoch: 309 cost = 0.00515605
Epoch: 310 cost = 0.00445717
Epoch: 311 cost = 0.00417656
Epoch: 312 cost = 0.00400142
Epoch: 313 cost = 0.00384786
Epoch: 314 cost = 0.00382334
Epoch: 315 cost = 0.00378541
Epoch: 316 cost = 0.00365881
Epoch: 317 cost = 0.00359201
Epoch: 318 cost = 0.00361274
Epoch: 319 cost = 0.00361702
Epoch: 320 cost = 0.00357063
Epoch: 321 cost = 0.00354429
Epoch: 322 cost = 0.00362382
Epoch: 323 cost = 0.00364289
Epoch: 324 cost = 0.00363717
Epoch: 325 cost = 0.00371851
Epoch: 326 cost = 0.00384121
Epoch: 327 cost = 0.00395767
Epoch: 328 cos

Epoch: 691 cost = 0.00058324
Epoch: 692 cost = 0.00056375
Epoch: 693 cost = 0.00058960
Epoch: 694 cost = 0.00062984
Epoch: 695 cost = 0.00059951
Epoch: 696 cost = 0.00054374
Epoch: 697 cost = 0.00059010
Epoch: 698 cost = 0.00054811
Epoch: 699 cost = 0.00054377
Epoch: 700 cost = 0.00056282
Epoch: 701 cost = 0.00054051
Epoch: 702 cost = 0.00057979
Epoch: 703 cost = 0.00052164
Epoch: 704 cost = 0.00051214
Epoch: 705 cost = 0.00058613
Epoch: 706 cost = 0.00064833
Epoch: 707 cost = 0.00062281
Epoch: 708 cost = 0.00067060
Epoch: 709 cost = 0.00077902
Epoch: 710 cost = 0.08365847
Epoch: 711 cost = 0.00291180
Epoch: 712 cost = 0.00059104
Epoch: 713 cost = 0.00051062
Epoch: 714 cost = 0.00049442
Epoch: 715 cost = 0.00052990
Epoch: 716 cost = 0.00047308
Epoch: 717 cost = 0.00050735
Epoch: 718 cost = 0.00055664
Epoch: 719 cost = 0.00054339
Epoch: 720 cost = 0.00055895
Epoch: 721 cost = 0.00053384
Epoch: 722 cost = 0.00053915
Epoch: 723 cost = 0.00051986
Epoch: 724 cost = 0.00056841
Epoch: 725 cos

In [34]:
# on 1000 epoch
print(sess.run(accuracy, feed_dict={X: X_test, Y: y_test}))

0.7011494
